In [ ]:
!pip install -q einops
!pip install -q datasets
!pip install -q bitsandbytes
!pip install -q accelerate
!pip install -q transformers
!pip install -q peft
!pip install -q wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.6 MB/s eta 0:00:00


In [ ]:
import wandb
import os

import pickle

import json

import torch

from tqdm import tqdm

from datasets import load_dataset

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training
)

In [ ]:
import os

import torch

SEED = 42
torch.manual_seed(SEED)

os.environ['WANDB_SILENT'] = 'true'

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

Поскольку моя видеокарта не тянет тяжелые модели, обучение поводил в колабе. Загружаю предобработанный датасет

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def load_data(filename):
    with open(filename, 'rb') as file:
        loaded_data = pickle.load(file)
        return loaded_data

In [ ]:
dataset = load_data('/content/drive/MyDrive/MIPT_NLPGener/HW_2/data/data.pkl')

print(dataset[0])
print(dataset[1])
print(f'Lenght of data: {len(dataset)}')

[" there's nothing to tell! he's just some guy i work with! c'mon, you're going out with the guy! there's gotta be something wrong with him!", 'so does he have a hump? a hump and a hairpiece?', " just, 'cause, i don't want her to go through what i went through with carl- oh! okay, everybody relax. this is not even a date. it's just two people going out to dinner and- not having s*x.", 'sounds like a date to me.']
[" alright, so i'm back in high school, i'm standing in the middle of the cafeteria, and i realise i am totally naked. oh, yeah. had that dream.", "then i look down, and i realise there's a phone... there.", ' instead of...?', "that's right.", ' never had that dream. no.', "all of a sudden, the phone starts to ring. and it turns out it's my mother, which is very weird, because- she never calls me!"]
Lenght of data: 5516


Преобразую его в формат диалога:

- реплика

- ответ_персонажа

- реплика

- ответ_персонажа

In [ ]:
fixed_dataset = []

for idx, data in enumerate(tqdm(dataset)):
    curr_text = []
    for message in data:
      curr_text.append(str(message))
    fixed_dataset.append({'number': idx, 'messages': curr_text})

100%|██████████| 5516/5516 [00:00<00:00, 199308.93it/s]


In [ ]:
fixed_dataset[0]

{'number': 0,
 'messages': [" there's nothing to tell! he's just some guy i work with! c'mon, you're going out with the guy! there's gotta be something wrong with him!",
  'so does he have a hump? a hump and a hairpiece?',
  " just, 'cause, i don't want her to go through what i went through with carl- oh! okay, everybody relax. this is not even a date. it's just two people going out to dinner and- not having s*x.",
  'sounds like a date to me.']}

In [ ]:
fixed_dataset[1]

{'number': 1,
 'messages': [" alright, so i'm back in high school, i'm standing in the middle of the cafeteria, and i realise i am totally naked. oh, yeah. had that dream.",
  "then i look down, and i realise there's a phone... there.",
  ' instead of...?',
  "that's right.",
  ' never had that dream. no.',
  "all of a sudden, the phone starts to ring. and it turns out it's my mother, which is very weird, because- she never calls me!"]}

Для простоты дальнейшей обработки сохраняю в json и сразу загружаю с помощью метода из datasets

In [ ]:
with open("dataset.json", "w") as f:
    json.dump(fixed_dataset, f, indent=4)

print("Dataset size:", len(fixed_dataset))

Dataset size: 5516


In [ ]:
dataset  = load_dataset("json", data_files="dataset.json")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset = dataset["train"].train_test_split(test_size=0.1)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['number', 'messages'],
        num_rows: 4964
    })
    test: Dataset({
        features: ['number', 'messages'],
        num_rows: 552
    })
})

Модель выбирал исходя из размеров, чтобы была возможность обучить ее на доступной в колабе T4.

In [ ]:
BASE_MODEL = 'microsoft/phi-2'

bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type='nf4',
                                bnb_4bit_compute_dtype='float16',
                                bnb_4bit_use_double_quant=True)

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map='auto',
    quantization_config=bnb_config,
    trust_remote_code=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=False,
                                          add_eos_token=True,
                                          trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.truncation_side = "left"

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

In [ ]:
# Гиперпараметры LoRA
LORA_R = 64
LORA_ALPHA = 64
LORA_DROPOUT = 0.05

LORA_MODULES_TO_SAVE = ["lm_head", "embed_tokens"]
LORA_TARGET_MODULES = [ "q_proj", "k_proj", "v_proj", "dense" ]

In [ ]:
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules = LORA_TARGET_MODULES,
    modules_to_save = LORA_MODULES_TO_SAVE,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)

model.config.use_cache = False

model.print_trainable_parameters()

trainable params: 304,138,240 || all params: 3,083,822,080 || trainable%: 9.862379609137502


При обработке вводим подсказку-инструкцию с тегом ###system чтобы снизить галюцинирование результатов

In [ ]:
import os
from functools import partial

MAX_LENGTH = 512

templates = [
    " ###question: {msg} ", # реплики других персонажей
    " ###answer: {msg} "    # реплика выбранного персонажа
]

system_msg = "###system: answer the question as Chandler"
system_tokenized = tokenizer(
    system_msg,
    truncation=True,
    add_special_tokens=False)

IGNORE_INDEX = -100 # Индекс для отметки токенов, игнорируемых при подсчете loss

def tokenize(input, max_length):
    input_ids, attention_mask, labels = [], [], []

    input_ids += system_tokenized["input_ids"]
    attention_mask += system_tokenized["attention_mask"]

    # loss рассчитывается только для сообщений выбранного пресонажа, остальные реплики в расчете loss игнорируются
    labels += [IGNORE_INDEX]*len(system_tokenized["input_ids"])

    for i, msg in enumerate(input["messages"]):

        isCharact = i%2!=0
        msg_chatml = templates[isCharact].format(msg=msg)

        msg_tokenized = tokenizer(
          msg_chatml,
          truncation=True,
          add_special_tokens=False)

        input_ids += msg_tokenized["input_ids"]
        attention_mask += msg_tokenized["attention_mask"]

        # loss рассчитывается только для сообщений выбранного пресонажа, остальные реплики в расчете loss игнорируются
        labels += [IGNORE_INDEX]*len(msg_tokenized["input_ids"]) if not isCharact else msg_tokenized["input_ids"]

    # обрезаем до заданной максимальной длины
    return {
        "input_ids": input_ids[:max_length],
        "attention_mask": attention_mask[:max_length],
        "labels": labels[:max_length],
    }

dataset_tokenized = dataset.map(
    partial(tokenize, max_length=MAX_LENGTH),
    batched = False,
    num_proc = os.cpu_count(),
    remove_columns = dataset["train"].column_names
)

Map (num_proc=2):   0%|          | 0/4964 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/552 [00:00<?, ? examples/s]

In [ ]:
def collate(elements):
    tokens = [e["input_ids"] for e in elements]
    tokens_maxlen = max([len(t) for t in tokens])

    for i, sample in enumerate(elements):
        input_ids = sample["input_ids"]
        labels = sample["labels"]
        attention_mask = sample["attention_mask"]

        pad_len = tokens_maxlen-len(input_ids)

        input_ids.extend( pad_len * [tokenizer.pad_token_id] )
        labels.extend( pad_len * [IGNORE_INDEX] )
        attention_mask.extend( pad_len * [0] )

    batch={
        "input_ids": torch.tensor( [e["input_ids"] for e in elements] ),
        "labels": torch.tensor( [e["labels"] for e in elements] ),
        "attention_mask": torch.tensor( [e["attention_mask"] for e in elements] ),
    }
    return batch

In [ ]:
val_interval = 1
best_metric = -1
best_metric_epoch = -1
wandb.login()

wandb.init(project="NLPGener_HW2")
wandb.run.name = "phi2_chat_friends_chand"
wandb.watch(model)

<IPython.core.display.Javascript object>

wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


[]

Гиперпараметры для обучения подбирал исходя из доступной в колабе памяти, стремления сохранить адекватные размер батча и время обучения и уменьшить галюцинирование результатов. Также, для уменьшения галюцинирования результата ввел weight_decay=0.01

In [ ]:
from transformers import TrainingArguments, Trainer

BATCH_SIZE = 4
GRAD_ACC_STEPS = 1
NUM_EPOCHS = 1
LEARNING_RATE = 1.0e-4

steps_per_epoch=len(dataset_tokenized["train"])//(BATCH_SIZE*GRAD_ACC_STEPS)

args = TrainingArguments(
    output_dir="phi2_chan",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=16,
    evaluation_strategy="steps",
    logging_steps=1,
    eval_steps=steps_per_epoch//2,
    save_steps=steps_per_epoch,
    save_total_limit=1,
    gradient_accumulation_steps=GRAD_ACC_STEPS,
    num_train_epochs=NUM_EPOCHS,
    lr_scheduler_type="constant",
    optim="paged_adamw_32bit",
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    warmup_ratio = 0.03,
    group_by_length=True,
    bf16=False,
    ddp_find_unused_parameters=False,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=collate,
    train_dataset=dataset_tokenized["train"],
    eval_dataset=dataset_tokenized["test"],
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
620,1.847100,1.941620
1240,2.351700,1.643182


TrainOutput(global_step=1241, training_loss=1.9866238668212768, metrics={'train_runtime': 5888.5891, 'train_samples_per_second': 0.843, 'train_steps_per_second': 0.211, 'total_flos': 1.407735194112e+16, 'train_loss': 1.9866238668212768, 'epoch': 1.0})

In [ ]:
from huggingface_hub import login

login()

In [ ]:
trainer.push_to_hub()

training_args.bin:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

events.out.tfevents.1709306648.defde22193be.408.0:   0%|          | 0.00/267k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/yurezsml/phi2_chan/commit/5433706e3293cdf931cf88dcb95dfbb58b172ede', commit_message='End of training', commit_description='', oid='5433706e3293cdf931cf88dcb95dfbb58b172ede', pr_url=None, pr_revision=None, pr_num=None)